In [7]:
import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

In [15]:
data = {}

dbtable = 'z3dmc.meb_dgm'
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [24]:
try:
   sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
   engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
   dmc_sql = "select * from "+dbtable+" where rownum<100"
   df_dmc = pd.read_sql(dmc_sql, engine)
except SQLAlchemyError as e:
   print(e)

In [26]:
df_dmc.shape

(99, 30)

In [27]:
df_dmc.head()

,id,dmc,nr_dgm,status,czas_fazy_1,czas_fazy_2,czas_fazy_3,max_predkosc,cisnienie_tloka,cisnienie_koncowe,...,vds_vac_hose1,vds_vac_hose2,vds_vac_tank,vds_vac_valve1,vds_vac_valve2,czas_taktu,data_znakowania,data_odlania,metal_level,metal_pressure
0,220,190627091611011003610,10,4,1850,88,47.5,6.52,7,264,...,None,None,None,None,None,None,None,None,None,None
1,3416,190719071611011006077,10,1,2197,89,46.5,6.59,11,285,...,None,None,None,None,None,None,None,None,None,None
2,3417,190719071811011006078,10,1,2184,86,45.0,6.52,8,284,...,None,None,None,None,None,None,None,None,None,None
3,3418,190719072011011006079,10,1,2159,89,45.5,6.54,6,284,...,None,None,None,None,None,None,None,None,None,None
4,3419,190719072111011006080,10,1,2175,89,47.0,6.57,8,283,...,None,None,None,None,None,None,None,None,None,None


In [11]:
connection=cx_Oracle.connect(username,password,dsn,encoding='UTF-8')
connection.version
connection.close()

In [12]:
try:
    with cx_Oracle.connect(username,password,dsn,encoding='UTF-8') as connection:
        with connection.cursor() as cursor:
            cursor.execute(dmc_sql)
            while True:
                row = cursor.fetchone()
                if row is None:
                    break
                print(row)
                data={'id':row[0],'dmc':row[1]}
    cursor.close()
    connection.close()
except cx_Oracle.Error as error:
    print(error)    

(2071, '0MH301103CVW41910090437321102S111910040418', 1, datetime.datetime(2019, 10, 9, 8, 33, 13), datetime.datetime(2019, 11, 6, 6, 49, 5), 504, datetime.datetime(2019, 11, 6, 6, 49, 4), None, None, 700)
(2070, '0MH301103CVW41910090437211102S111910040342', 2, datetime.datetime(2019, 10, 9, 8, 32, 53), datetime.datetime(2019, 11, 6, 9, 14, 37), 366, datetime.datetime(2019, 11, 6, 9, 14, 36), None, None, 700)
(2069, '0MH301103CVW41910090437121102S111910040330', 107, datetime.datetime(2019, 10, 9, 8, 32, 26), datetime.datetime(2019, 11, 12, 6, 32, 27), 366, datetime.datetime(2019, 11, 12, 6, 32, 26), None, None, 700)
(2068, '0MH301103CVW41910090437011102S111910040351', 1, datetime.datetime(2019, 10, 9, 8, 18, 26), datetime.datetime(2019, 11, 6, 5, 26, 55), 504, datetime.datetime(2019, 11, 6, 5, 26, 54), None, None, 700)
(2067, '0MH301103CVW41910090436721102S111910040345', 2, datetime.datetime(2019, 10, 9, 8, 18, 5), datetime.datetime(2019, 10, 9, 8, 22, 31), 0, None, None, None, None)
(2